In [1]:
datapath = "/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct.tsv"
# datapath = "/content/GPT-4o_data.tsv"


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -q lens-metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 104.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.4/891.4 kB 44.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incomp

In [ ]:
!pip install -q evaluate
!pip install sacremoses sacrebleu
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.6 MB/s eta 0:00:00
   ━

In [4]:
import pandas as pd
import json
import torch


In [ ]:
complex_filepath = '/content/LLaMa 3.1 70B instruction (2nd run)'
simple_filepath = '/content/test.json'

In [ ]:
import json
with open(complex_filepath,'r', encoding='utf-8') as f:
  complex = json.load(f)
with open(simple_filepath,'r', encoding='utf-8') as s:
  simple = json.load(s)

In [ ]:
complex_text = []
simple_text = []
for q_key, q_data in complex.items():
  for abstract_key, abstract_data in q_data.get("abstracts", {}).items():
    sentences = abstract_data.get("sentences", [])
    complex_text += sentences

for q_key, q_data in simple.items():
  for abstract_key, abstract_data in q_data.get("abstracts", {}).items():
    sentences = abstract_data.get("sentences", [])
    simple_text += sentences

In [ ]:
assert len(complex_text) == len(simple_text)
for a, b in zip(complex_text, simple_text):
  if a == "":
    print(a)
  if b == "":
    print(b)

In [5]:
from lens import download_model, LENS, LENS_SALSA
lens_salsa_path = download_model("davidheineman/lens-salsa") # see https://huggingface.co/davidheineman/lens-salsa
lens_salsa = LENS_SALSA(lens_salsa_path)

complex_text = [
    "They are culturally akin to the coastal peoples of Papua New Guinea."
]
simple_text = [
    "They are culturally similar to the people of Papua New Guinea."
]

scores, word_level_scores = lens_salsa.score(complex_text, simple_text, batch_size=8, devices=[0])
print(scores) # [72.40909337997437]

# LENS-SALSA also returns an error-identification tagging, recover_output() will return the tagged output
tagged_output = lens_salsa.recover_output(word_level_scores, threshold=0.5)
print(tagged_output)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:165: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]


[72.40909337997437]
[{'src_tagged': 'they are culturally akin to the coastal peoples of papua new guinea.', 'mt_tagged': 'they are culturally similar to the people of papua new guinea.'}]


In [6]:
from lens import download_model, LENS

lens_path = download_model("davidheineman/lens")

# Original LENS is a real-valued number.
# Rescaled version (rescale=True) rescales LENS between 0 and 100 for better interpretability.
# You can also use the original version using rescale=False
lens = LENS(lens_path, rescale=True)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model.ckpt:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/705 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

KeyboardInterrupt: 

In [7]:
df = pd.read_csv(datapath, sep='\t')
df[:5]

,Abstract ID,Abstract Sentence,adaptation2,adaptation3,adaptation1,generated
0,15902691,Muscle cramps are a common problem characteriz...,Muscle cramps are a common problem represented...,NaN,NaN,Muscle cramps happen when a muscle tightens up...
1,15902691,"These true cramps, which originate from periph...","These true cramps, coming from nerves outside ...",NaN,NaN,"These true cramps, which start from nerves out..."
2,15902691,"Medical history, physical examination, and a l...","Medical history, physical check-up, and lab sc...",NaN,NaN,To figure out why you're getting muscle cramps...
3,15902691,"Despite the ""benign"" nature of cramps, many pa...","Despite their harmless nature, cramps are unco...",NaN,NaN,Although muscle cramps are generally not serio...
4,15902691,Treatment options are guided both by experienc...,Experience and limited medical studies guide t...,NaN,NaN,We choose treatment options based on what has ...


In [8]:
complex = df["Abstract Sentence"].to_list()
simple = df["generated"].to_list()

# merged_adaptations = df[['adaptation1', 'adaptation2', 'adaptation3']].apply(lambda x: [i for i in x if pd.notna(i)], axis=1).tolist()
# merged_adaptations

In [10]:
def process_sequences(sequences, max_length):
    processed_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            # Truncate if sequence is too long
            processed_sequences.append(seq[:max_length])
        else:
            # Pad if sequence is too short
            processed_sequences.append(seq)  # Pad with empty strings
    return processed_sequences

# Define maximum sequence length
max_length = 512  # Or any suitable value based on your model and data

complex = process_sequences(complex, max_length)
simple = process_sequences(simple, max_length)

scores, word_level_scores = lens_salsa.score(complex, simple, batch_size=8, devices=[0])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

Evaluating: 0it [00:00, ?it/s]
Predicting DataLoader 0: 100%|██████████| 952/952 [04:39<00:00,  3.41it/s]


In [12]:
def calculate_average(numbers):
    if not numbers:  # 检查列表是否为空
        return 0
    return sum(numbers) / len(numbers)
calculate_average(scores)

73.89415828429145

In [ ]:
import numpy as np
import scipy.stats as stats

mean = np.mean(scores)
std_dev = np.std(scores,ddof=1)
n = len(scores)
t_score = stats.t.ppf(0.975, df=n-1)  # 0.975 for 95% CI (two-tailed)
standard_error = std_dev / np.sqrt(n)
margin_of_error = t_score * standard_error
print(f"The 95% confidence interval is: ({mean}+-{margin_of_error})")


The 95% confidence interval is: (74.27848633927445+-0.1696849619568459)


In [ ]:
references = []
for adp1,adp2,adp3 in zip(df["adaptation1"].to_list(),df["adaptation2"].to_list(),df["adaptation3"].to_list()):
  tmp = [x for x in [adp1,adp2,adp3] if pd.notna(x)]
  if len(tmp) == 0:
    tmp = ['']
  references.append(tmp)

In [ ]:
assert len(complex) == len(simple) == len(references)
references[:5]

[['Muscle cramps are a common problem represented by sudden, painful, involuntary muscle contractions.'],
 ['These true cramps, coming from nerves outside the brain and spinal cord, may be identifiable from other muscle pains.'],
 ['Medical history, physical check-up, and lab screenings help determine different causes of muscle cramps.'],
 ['Despite their harmless nature, cramps are uncomfortable for many.'],
 ['Experience and limited medical studies guide treatment.']]

In [ ]:
scores = lens.score(complex, simple, references, batch_size=8, devices=[0])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 1154/1154 [04:55<00:00,  3.90it/s]


In [ ]:
scores[:5]

[63.3679202084908,
 79.11664522252124,
 77.75983271491286,
 79.80933807549245,
 63.71860538775104]

In [ ]:
df['LENS'] = scores
df[:5]


,Abstract Sentence,adaptation1,adaptation2,adaptation3,generated,LENS
0,Muscle cramps are a common problem characteriz...,NaN,Muscle cramps are a common problem represented...,NaN,Muscle cramps happen when a muscle suddenly an...,63.367920
1,"These true cramps, which originate from periph...",NaN,"These true cramps, coming from nerves outside ...",NaN,These real cramps come from nerve problems and...,79.116645
2,"Medical history, physical examination, and a l...",NaN,"Medical history, physical check-up, and lab sc...",NaN,"Learning about your medical history, doing a p...",77.759833
3,"Despite the ""benign"" nature of cramps, many pa...",NaN,"Despite their harmless nature, cramps are unco...",NaN,"Even though cramps aren't dangerous, many peop...",79.809338
4,Treatment options are guided both by experienc...,NaN,Experience and limited medical studies guide t...,NaN,Treatments are based on both past experiences ...,63.718605


In [ ]:
df['LENS'].mean()

64.75086599899707

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct_lens.tsv', sep='\t', index=False)

In [ ]:
from evaluate import load
sari = load("sari")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
A = [
    "Muscle cramps are a common problem characterized by a sudden, painful, involuntary contraction of muscle.",
    "asdasdasd"
]
B = [
    "Muscle cramps happen when a muscle tightens up suddenly and hurts, and you can't control it.",
    "as as as"
]
C = [[
    "Muscle cramps are a common problem represented by sudden, painful, involuntary muscle contractions."],
     ["asdasd"],
     ]

In [ ]:
sari_scores = sari.compute(sources=A, predictions=B, references=C)
sari_scores

{'sari': 49.079861111111114}

In [ ]:
sari_scores = []

for com, simp, renf in zip(complex, simple, references):
  com = [com]
  simp = [simp]
  renf = [renf]
  # print(com)
  # print(simp)
  # print(renf)
  sari_score = sari.compute(sources=com, predictions=simp, references=renf)
  sari_scores.append(sari_score)
  # print(sari_score)


In [ ]:
assert len(complex) == len(simple) == len(references) == len(sari_scores)

In [ ]:
# df['LENS'] = scores
df['SARI'] = [score['sari'] for score in sari_scores]
df[:5]

,Abstract Sentence,adaptation1,adaptation2,adaptation3,generated,SARI
0,Muscle cramps are a common problem characteriz...,NaN,Muscle cramps are a common problem represented...,NaN,Muscle cramps happen when a muscle suddenly an...,22.545687
1,"These true cramps, which originate from periph...",NaN,"These true cramps, coming from nerves outside ...",NaN,These real cramps come from nerve problems and...,36.692782
2,"Medical history, physical examination, and a l...",NaN,"Medical history, physical check-up, and lab sc...",NaN,"Learning about your medical history, doing a p...",46.473282
3,"Despite the ""benign"" nature of cramps, many pa...",NaN,"Despite their harmless nature, cramps are unco...",NaN,"Even though cramps aren't dangerous, many peop...",45.535714
4,Treatment options are guided both by experienc...,NaN,Experience and limited medical studies guide t...,NaN,Treatments are based on both past experiences ...,51.792929


In [ ]:
df['SARI'].mean()

37.72345767438335

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct_sari.tsv', sep='\t', index=False)

In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
bertscore_scores = []

ref_list = [ref[0] for ref in references]



In [ ]:
# bertscore_scores = []

# simple_text_len_rent = []
# for simp, renf in zip(simple, references):
#   simple_text_len_rent.append([simp]*len(renf))

# for i,j in zip(simple_text_len_rent, references):
#   assert len(i)==len(j)

# print(len(references))
# print(len(simple_text_len_rent))

# ref_list = []
# for i in references:
#   ref_list+=i
# pred_list = []
# for i in simple_text_len_rent:
#   pred_list += i

# print(len(ref_list))
# print(len(pred_list))

In [ ]:
bertscores = bertscore.compute(predictions=simple, references=ref_list, lang="en")

In [ ]:
bertscores.keys()

dict_keys(['precision', 'recall', 'f1', 'hashcode'])

In [ ]:
df['BERTScore_f1'] = bertscores['f1']
df['BERTScore_precision'] = bertscores['precision']
df['BERTScore_recall'] = bertscores['recall']

In [ ]:
df[:5]

,Abstract ID,Abstract Sentence,adaptation2,adaptation3,adaptation1,generated,BERTScore_f1,BERTScore_precision,BERTScore_recall
0,15902691,Muscle cramps are a common problem characteriz...,Muscle cramps are a common problem represented...,NaN,NaN,Muscle cramps happen when a muscle tightens up...,0.915614,0.909722,0.921583
1,15902691,"These true cramps, which originate from periph...","These true cramps, coming from nerves outside ...",NaN,NaN,"These true cramps, which start from nerves out...",0.911643,0.864881,0.963751
2,15902691,"Medical history, physical examination, and a l...","Medical history, physical check-up, and lab sc...",NaN,NaN,To figure out why you're getting muscle cramps...,0.911069,0.901420,0.920926
3,15902691,"Despite the ""benign"" nature of cramps, many pa...","Despite their harmless nature, cramps are unco...",NaN,NaN,Although muscle cramps are generally not serio...,0.878835,0.842450,0.918504
4,15902691,Treatment options are guided both by experienc...,Experience and limited medical studies guide t...,NaN,NaN,We choose treatment options based on what has ...,0.883175,0.872827,0.893771


In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct_bertscore.tsv', sep='\t', index=False)
# df.to_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_gpt4_bertscore.tsv', sep='\t', index=False)

In [ ]:
output = []
for _, row in df.iterrows():
    # Collect non-empty adaptation fields
    references = [row['adaptation1'], row['adaptation2'], row['adaptation3']]
    references = [ref for ref in references if pd.notna(ref) and ref != '']
    # scores = lens.score(row['Abstract Sentence'], row['generated'], references, batch_size=8, devices=[0])
    entry = {
        "complex": row['Abstract Sentence'],
        "simple": row['generated'],
        "references": references
    }
    output.append(entry)
    # print(entry)

In [ ]:
bleu = load("bleu")
bleu_scores = bleu.compute(predictions=simple, references=references)
bleu_scores

{'bleu': 0.056288188773965186,
 'precisions': [0.21262741342594232,
  0.07538483345145287,
  0.035009673776109126,
  0.017888659488889885],
 'brevity_penalty': 1.0,
 'length_ratio': 2.6757380579678354,
 'translation_length': 536737,
 'reference_length': 200594}

In [ ]:
df['BLEU'] = bleu_scores['bleu']

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct_lens_sari_bertscore_bleu.tsv', sep='\t', index=False)

In [ ]:
output_json = json.dumps(output, indent=4)

# Display the processed output
output_json

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
output_file_path = '/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/output_formatted.json'
with open(output_file_path, 'w') as f:
    f.write(output_json)

In [ ]:
df.read_csv('/content/drive/MyDrive/Colab Notebooks/PLABA_2024-Task_2/plaba_generated_70b_instruct_lens_sari_bertscore_bleu.tsv', sep='\t',)
